In [1]:
import pandas as pd

dataset = pd.read_csv('./data/steam-store-games/steam.csv')
descriptions = pd.read_csv('./data/steam-store-games/steam_description_data.csv')

dataset = dataset.set_index('appid').join(descriptions.set_index('steam_appid'))

dataset.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
appid,,,,,,,,,,,,,,,,,,,,
10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [2]:
import re
import string

html_cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
remove_punctuation_table = str.maketrans(dict.fromkeys(string.punctuation))

def tokenize(text):
    text = re.sub(html_cleanr, '', text)
    text = text.translate(remove_punctuation_table)
    text = text.lower()
    return text.split()

In [3]:
appids = dataset.index.values
names = dataset['name'].values
corpus = dataset['detailed_description'].values
tokenized_corpus = [tokenize(doc) for doc in corpus]

In [4]:
from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(tokenized_corpus)

In [5]:
import numpy as np

def recommend_single(query_game, N=10):
    games = dataset[dataset['name'] == query_game]
    if len(games) == 0:
        raise ValueError(f'Unknown game name `{query_game}`')
    query = games['detailed_description'].iloc[0]
    scores = bm25.get_scores(tokenize(query))
    # scores *= weights
    top_n = np.argsort(scores)[::-1][:N]
    for i in top_n:
        print(f'https://store.steampowered.com/app/{appids[i]:<6}\t{names[i]}')

In [6]:
recommend_single("PLAYERUNKNOWN'S BATTLEGROUNDS")

https://store.steampowered.com/app/578080	PLAYERUNKNOWN'S BATTLEGROUNDS
https://store.steampowered.com/app/931250	Pixel Royale
https://store.steampowered.com/app/658470	Mini Battlegrounds
https://store.steampowered.com/app/474790	BATTLE PIXEL'S SURVIVAL GROUND
https://store.steampowered.com/app/840130	Battle Royale: Survivors 究极求生:大逃杀
https://store.steampowered.com/app/769220	TRAFICO
https://store.steampowered.com/app/903070	War.io : Survival Battle Royale
https://store.steampowered.com/app/857240	The Prison Experiment: Battle Royale
https://store.steampowered.com/app/433850	Z1 Battle Royale
https://store.steampowered.com/app/912300	War.io : Zombie Battle Royale


In [7]:
recommend_single("Counter-Strike")

https://store.steampowered.com/app/10    	Counter-Strike
https://store.steampowered.com/app/204560	Confrontation
https://store.steampowered.com/app/939000	Crisis VRigade
https://store.steampowered.com/app/32710 	Armored Fist 3
https://store.steampowered.com/app/576940	Hunt For Gods
https://store.steampowered.com/app/39000 	Moonbase Alpha
https://store.steampowered.com/app/627450	RUGBY 18
https://store.steampowered.com/app/13540 	Tom Clancy's Rainbow Six® Vegas
https://store.steampowered.com/app/240320	Panzar
https://store.steampowered.com/app/96200 	Steel Storm: Burning Retribution


In [8]:
recommend_single("Half-Life 2")

https://store.steampowered.com/app/220   	Half-Life 2
https://store.steampowered.com/app/500   	Left 4 Dead
https://store.steampowered.com/app/259170	Alone in the Dark
https://store.steampowered.com/app/279920	Infinity Runner
https://store.steampowered.com/app/601050	Attack on Titan 2 - A.O.T.2 - 進撃の巨人２
https://store.steampowered.com/app/34870 	Sniper: Ghost Warrior 2
https://store.steampowered.com/app/408410	X-Morph: Defense
https://store.steampowered.com/app/236370	Interstellar Marines
https://store.steampowered.com/app/362890	Black Mesa
https://store.steampowered.com/app/513320	ALICE VR


In [9]:
recommend_single("Star Wars: Battlefront 2 (Classic, 2005)")

https://store.steampowered.com/app/6060  	Star Wars: Battlefront 2 (Classic, 2005)
https://store.steampowered.com/app/32510 	LEGO® Star Wars™ III - The Clone Wars™
https://store.steampowered.com/app/6020  	STAR WARS™ Jedi Knight - Jedi Academy™
https://store.steampowered.com/app/32430 	STAR WARS™ - The Force Unleashed™ Ultimate Sith Edition
https://store.steampowered.com/app/32470 	STAR WARS™ Empire at War - Gold Pack
https://store.steampowered.com/app/32500 	STAR WARS™: The Force Unleashed™ II
https://store.steampowered.com/app/6030  	STAR WARS™ Jedi Knight II - Jedi Outcast™
https://store.steampowered.com/app/32380 	STAR WARS™ Jedi Knight: Dark Forces II
https://store.steampowered.com/app/356500	STAR WARS™ Galactic Battlegrounds Saga
https://store.steampowered.com/app/456540	STAR WARS™: Rebel Assault I + II


In [10]:
recommend_single("LEGO® Batman™: The Videogame")

https://store.steampowered.com/app/21000 	LEGO® Batman™: The Videogame
https://store.steampowered.com/app/208650	Batman™: Arkham Knight
https://store.steampowered.com/app/209000	Batman™: Arkham Origins
https://store.steampowered.com/app/24200 	DC Universe™ Online
https://store.steampowered.com/app/200260	Batman: Arkham City - Game of the Year Edition
https://store.steampowered.com/app/213330	LEGO® Batman™ 2: DC Super Heroes
https://store.steampowered.com/app/313690	LEGO® Batman™ 3: Beyond Gotham
https://store.steampowered.com/app/206210	Gotham City Impostors Free to Play
https://store.steampowered.com/app/35140 	Batman: Arkham Asylum Game of the Year Edition
https://store.steampowered.com/app/502820	Batman™: Arkham VR


In [11]:
cache = {}


def recommend(query_games):
    scores = np.zeros(len(corpus))
    for query_game in query_games:
        games = dataset[dataset["name"] == query_game]
        if len(games) == 0:
            raise ValueError(f"Unknown game name `{query_game}`")
        game_id = games["name"].iloc[0]
        if game_id not in cache:
            query = games["detailed_description"].iloc[0]
            game_scores = bm25.get_scores(tokenize(query))
            cache[game_id] = game_scores
        else:
            game_scores = cache[game_id]
        scores += game_scores
    top_n = np.argsort(scores)[::-1][3:]
    games = []
    for i in top_n:
        games.append(names[i])
    return games


In [12]:
recommend(["PLAYERUNKNOWN'S BATTLEGROUNDS", "Counter-Strike"])[:10]


['Mini Battlegrounds',
 'Pixel Royale',
 'Battle Royale: Survivors 究极求生:大逃杀',
 'War.io : Zombie Battle Royale',
 'War.io : Survival Battle Royale',
 'Z1 Battle Royale',
 "BATTLE PIXEL'S SURVIVAL GROUND",
 'Burndown',
 'Survival Games',
 'Cuisine Royale']